In [1]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain.document_loaders import WebBaseLoader
import os

llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")

def search_tool_wrapper(search_function):
    def tool(query):
        return search_function(query)
    return tool

def search_wikipedia(query):
    wikipedia = WikipediaAPIWrapper()
    return wikipedia.run(query)

wikipedia_tool = Tool(
    name="WikipediaSearch",
    func=search_tool_wrapper(search_wikipedia),
    description="Search Wikipedia for relevant information about a given topic"
)

def search_duckduckgo(query):
    duckduckgo = DuckDuckGoSearchAPIWrapper()
    return duckduckgo.run(query)

duckduckgo_tool = Tool(
    name="DuckDuckGoSearch",
    func=search_tool_wrapper(search_duckduckgo),
    description="Search DuckDuckGo for web links related to a given topic"
)

def scrape_website(url):
    loader = WebBaseLoader(url)
    docs = loader.load()
    return "\n".join([doc.page_content for doc in docs])

scraper_tool = Tool(
    name="WebScraper",
    func=scrape_website,
    description="Scrape content from a given website URL"
)

def save_to_file(content):
    directory = os.path.dirname("./research_results.txt")
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    with open("./research_results.txt", "w", encoding="utf-8") as file:
        file.write(content)
    return "Results saved to research_results.txt"

save_tool = Tool(
    name="SaveResults",
    func=save_to_file,
    description="Scrape content saved to research_results.txt"
)

def create_research_agent():
    agent = initialize_agent(
        llm=llm,
        verbose=True,
        agent=AgentType.OPENAI_FUNCTIONS,
        handle_parsing_errors=True,  
        tools=[wikipedia_tool, duckduckgo_tool, scraper_tool, save_tool],
    )
    return agent


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\lesun\AppData\Local\Temp\ipykernel_22124\2509221162.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")


In [4]:
agent = create_research_agent()
response = agent.run("Research about the XZ backdoor")
save_to_file(response)
print(response)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `XZ backdoor`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Ut

In [5]:
if os.path.exists("./research_results.txt"):
    print("파일이 존재합니다.")
else:
    print("파일이 존재하지 않습니다. 먼저 데이터를 저장하세요.")

파일이 존재합니다.
